# Specviz Demonstration Notebook
This notebook demonstrates the Specviz API in the Notebook setting. UI equivalents for these actions, as well as additional documentation about Specviz, can be found here: https://jdaviz.readthedocs.io/en/latest/specviz/

## Setup

In [ ]:
import astropy.units as u

In [ ]:
# Suppress warnings
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

## Create Specviz via Helper

In [ ]:
from jdaviz import Specviz
specviz = Specviz()

## Display Specviz

Note: you probably want to pick one of the three options below depending on preference. Also not the latter two will only work in JupyterLab, not the "classic" notebook interface.

This option will show Specviz inline in the notebook

In [ ]:
specviz.app

This option will show specviz in a new tab on the right side of the notebook:

In [ ]:
specviz.show_in_sidecar()

This will open specviz in a new JupyterLab tab:

In [ ]:
specviz.show_in_new_tab()

## Load a file to Specviz

In [ ]:
from astropy.utils.data import download_file
fn = download_file('https://data.sdss.org/sas/dr14/sdss/spectro/redux/26/spectra/0751/spec-0751-52251-0160.fits', cache=True)
specviz.load_spectrum(fn, "myfile", format="SDSS-III/IV spec")

## Subset Retrieval
User Task: Select a subset in the viewer above

After defining regions in your spectra, you can retrieve your subsets through different means. To retrieve your subset by name:

In [ ]:
# Returns a version of the whole spectra, with a mask applied
specviz.get_spectra('Subset 1')

Or, if you've defined multiple regions, you can retrieve all defined regions/subsets via:

In [ ]:
specviz.app.get_subsets_from_viewer('spectrum-viewer')

## Screenshot Saving

In [ ]:
specviz.app.get_viewer("spectrum-viewer").figure.save_png()

## Panning/Zooming in Specviz
### Limit Methods
You can use the methods x_limits() and y_limits() to modify the field of view of Specviz. You can provide it a scalar (which assumes the units of the loaded spectra), an Astropy Quantity, or 'auto' to automatically scale.

In [ ]:
specviz.x_limits()
specviz.x_limits(650*u.nm,750*u.nm)
specviz.y_limits('auto', 110.0)

### Disable Scientific Notation
Scientific notation is used in the axes of the viewers by default. To deactivate it, run the following code:

In [ ]:
# axis 1 corresponds to the Y-axis and axis 0 to the X-axis
# fmt can be '0.1e' to set scientific notation or '0.2f' to turn it off
specviz.set_spectrum_tick_format(fmt="0.2f", axis=1)

### Autoscale Methods
You can also quickly return to the default zoom using autoscale_x() and autoscale_y()

In [ ]:
specviz.autoscale_x()
specviz.autoscale_y()

## Extracting Models
If models were added using the model fitting plugin, they can be extracted using the following property

In [ ]:
specviz.fitted_models

If the name of a particular model is known, it can be extracted from the fitted_models property

In [ ]:
specviz.fitted_models['Model']

Alternatively, the following getter can be used

In [ ]:
models = specviz.get_models()
models